In [2]:
import numpy as np
import networkx as nx
from scipy import sparse
import os

# Parameters (customizable; same as before)
n = 20          # Nodes
m = 2           # Dimensions
time = 500      # Time steps
transient = 14000  # Transient discard
beta = 4.1
mu = 0.001
sigma = 0.001
C = 0.1
gamma = 0.001
eta = 0.2

# Network generation
def network_generate(n, eta):
    G = nx.scale_free_graph(n, alpha=0.2, beta=0.3, gamma=0.5)
    G.remove_edges_from(nx.selfloop_edges(G))
    edges_to_remove = []
    seen = set()
    for edge in G.edges:
        if (edge[0], edge[1]) in seen:
            edges_to_remove.append(edge)
        else:
            seen.add((edge[0], edge[1]))
    G.remove_edges_from(edges_to_remove)
    for edge in G.edges(data=True):
        edge[2]["weight"] = np.random.uniform(1.0 - eta, 1.0 + eta)
    A = nx.adjacency_matrix(G).todense().T
    k_in = np.zeros(n)
    for node in range(n):
        if G.in_degree(node) > 0:
            k_in[node] = sum(data["weight"] for _, _, data in G.in_edges(node, data=True))
    L = np.diag(k_in) - A
    delta = np.max(k_in)
    return G, A, L, delta, k_in

G, A, L, delta, k_in = network_generate(n, eta)

# Data simulation
x_noise = np.zeros(n * m)
x_noise[::2] = 1  # Noise on u

def data_generate(n, m, transient, time, beta, mu, sigma, C, L, delta, gamma, x_noise):
    def rulkov_map(x):
        x = x.reshape(n, m).T
        return np.asarray([
            beta / (1 + x[0]**2) + x[1],
            x[1] - mu * x[0] - sigma
        ]).T.flatten()

    h = np.eye(m)
    h[1, 1] = 0

    def net_dynamics(x):
        return rulkov_map(x) - (C / delta) * sparse.kron(L, h).dot(x) + x_noise * gamma * np.random.uniform(-1, 1, n * m)

    x0 = np.random.uniform(0.0, 1.0, n * m)
    for _ in range(transient):
        x0 = rulkov_map(x0)
    for _ in range(transient):
        x0 = net_dynamics(x0)
    x = np.zeros((n * m, time))
    x[:, 0] = x0
    for t in range(time - 1):
        x[:, t + 1] = net_dynamics(x[:, t])
    return x.reshape(n, m, time)

x = data_generate(n, m, transient, time, beta, mu, sigma, C, L, delta, gamma, x_noise)

# Save to .npz with directory creation
output_dir = '../data'
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist
np.savez(os.path.join(output_dir, 'generated_data.npz'), 
         x=x, A=A, L=L, delta=delta, k_in=k_in, 
         G=nx.to_numpy_array(G),  # Convert Graph to adjacency matrix for saving
         params={
             'n': n, 'm': m, 'time': time, 'transient': transient,
             'beta': beta, 'mu': mu, 'sigma': sigma, 'C': C,
             'gamma': gamma, 'eta': eta
         })

print("Data saved to", os.path.join(output_dir, 'generated_data.npz'))

Data saved to ../data/generated_data.npz
